<a href="https://colab.research.google.com/github/nakshkeshav234/finalproject/blob/master/MTF%20CALCULATER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# STEP 1: Environment setup (run this once)

!pip -q install gradio pandas numpy matplotlib plotly

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import warnings, sys, os

warnings.filterwarnings("ignore")
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

def money(x):
    try:
        return f"₹{float(x):,.2f}"
    except:
        return "₹0.00"

print("✅ Libraries ready!")
for pkg in ["pandas","numpy","matplotlib","plotly","gradio"]:
    try:
        mod = __import__(pkg)
        ver = getattr(mod, "__version__", "OK")
        print(f"• {pkg} {ver}")
    except Exception as e:
        print(f"• {pkg} loaded")


✅ Libraries ready!
• pandas 2.2.2
• numpy 2.0.2
• matplotlib 3.10.0
• plotly 5.24.1
• gradio 5.42.0


In [2]:
# STEP 2: Core engine — MTFCalculator (Groww-style)

import pandas as pd
from datetime import datetime

class MTFCalculator:
    """
    Margin Trading Facility (MTF) calculator with Groww-style summaries.
    - Tracks: Paid by You (margin), Broker Funded (financed principal)
    - True per-day interest: Interest/day = Funded * (DayRate% / 100)
      Total interest = Interest/day * Days Financed
    - Works with CSV or ad-hoc trades
    """

    # ---- initialization ----
    def __init__(self, default_user_contribution_pct=40.0):
        # % of Buy Value paid by trader (rest is financed). Change if your broker differs.
        self.default_user_contribution_pct = float(default_user_contribution_pct)
        self.df = pd.DataFrame()
        self.summary_stats = {}

    # ---- data loading ----
    def load_data_from_csv(self, file_path):
        """
        Load a ledger CSV. If missing Groww fields, they will be back-filled.
        Required columns: Trade Date, Ticker, Quantity (Buy), Buy Price
        Optional: Current Price (LTP), Status, Sell Date, Sell Price, Quantity (Sell), MTF Day Rate (%)
        """
        try:
            self.df = pd.read_csv(file_path)
            # Normalize columns
            if 'Trade Date' not in self.df.columns or 'Ticker' not in self.df.columns:
                raise ValueError("CSV must contain at least 'Trade Date' and 'Ticker' columns.")
            # Types
            self.df['Trade Date'] = pd.to_datetime(self.df['Trade Date'], errors='coerce')
            if 'Sell Date' in self.df.columns:
                self.df['Sell Date'] = pd.to_datetime(self.df.get('Sell Date'), errors='coerce')

            # Safe defaults
            self.df['Quantity (Buy)'] = self.df.get('Quantity (Buy)', 0).fillna(0).astype(float)
            self.df['Buy Price'] = self.df.get('Buy Price', 0).fillna(0.0).astype(float)
            self.df['Buy Value'] = (self.df['Quantity (Buy)'] * self.df['Buy Price']).astype(float)

            self.df['Quantity (Sell)'] = self.df.get('Quantity (Sell)', 0).fillna(0).astype(float)
            self.df['Sell Price'] = self.df.get('Sell Price', 0.0).fillna(0.0).astype(float)
            self.df['Sell Value'] = (self.df['Quantity (Sell)'] * self.df['Sell Price']).astype(float)

            # Status / Remaining
            if 'Status' not in self.df.columns:
                self.df['Status'] = self.df['Quantity (Buy)'].gt(self.df['Quantity (Sell)']).map({True:'Open', False:'Closed'})
            self.df['Remaining Qty'] = (self.df['Quantity (Buy)'] - self.df['Quantity (Sell)']).clip(lower=0)

            # LTP fallback
            self.df['Current Price (LTP)'] = self.df.get('Current Price (LTP)', self.df['Buy Price']).fillna(self.df['Buy Price'])

            # Rates & contributions
            self.df['MTF Day Rate (%)'] = self.df.get('MTF Day Rate (%)', 0.04).fillna(0.04).astype(float)
            self.df['User Contribution %'] = self.df.get('User Contribution %', self.default_user_contribution_pct).fillna(self.default_user_contribution_pct).astype(float)

            # Back-fill Groww style splits
            self.df['Paid by You'] = self.df.get('Paid by You', self.df['Buy Value'] * (self.df['User Contribution %']/100.0))
            self.df['MTF Financed Principal'] = self.df.get('MTF Financed Principal', self.df['Buy Value'] - self.df['Paid by You'])

            # P/L containers
            for col, default in [
                ('Unrealized P/L', 0.0), ('Realized P/L', 0.0), ('Fees & Taxes', 0.0),
                ('Interest Per Day', 0.0), ('MTF Days Financed', 0), ('MTF Interest', 0.0),
                ('Net P/L', 0.0), ('ROI %', 0.0)
            ]:
                if col not in self.df.columns:
                    self.df[col] = default

            self.calculate_mtf_metrics()
            print(f"✅ Data loaded successfully! {len(self.df)} trades found.")
            return True
        except Exception as e:
            print(f"❌ Error loading data: {e}")
            return False

    # ---- add a trade ----
    def add_new_trade(self, trade_date, ticker, quantity, buy_price, current_price=None, mtf_day_rate=0.04, user_contribution_pct=None):
        ucp = self.default_user_contribution_pct if user_contribution_pct is None else float(user_contribution_pct)
        buy_value = float(quantity) * float(buy_price)
        paid_by_you = buy_value * (ucp/100.0)
        financed = buy_value - paid_by_you

        new = {
            'Trade Date': pd.to_datetime(trade_date),
            'Ticker': str(ticker).upper(),
            'Segment': 'MTF',
            'Status': 'Open',
            'Quantity (Buy)': float(quantity),
            'Buy Price': float(buy_price),
            'Buy Value': buy_value,
            'Quantity (Sell)': 0.0,
            'Sell Price': 0.0,
            'Sell Date': pd.NaT,
            'Sell Value': 0.0,
            'Remaining Qty': float(quantity),
            'Current Price (LTP)': float(current_price) if current_price not in (None, "") else float(buy_price),

            'Unrealized P/L': 0.0,
            'Realized P/L': 0.0,
            'Fees & Taxes': 0.0,

            # Groww-style fields
            'User Contribution %': float(ucp),
            'Paid by You': float(paid_by_you),
            'MTF Financed Principal': float(financed),
            'MTF Day Rate (%)': float(mtf_day_rate),
            'Interest Per Day': 0.0,

            'MTF Days Financed': 0,
            'MTF Interest': 0.0,
            'Net P/L': 0.0,
            'ROI %': 0.0,
            'Notes': ''
        }
        self.df = pd.concat([self.df, pd.DataFrame([new])], ignore_index=True)
        self.calculate_mtf_metrics()
        print(f"✅ Added new trade: {ticker} - {quantity} @ ₹{buy_price}")

    # ---- sell from an open position ----
    def sell_trade(self, ticker, sell_quantity, sell_price, sell_date=None):
        if sell_date is None:
            sell_date = datetime.now().date()

        mask = (self.df['Ticker'].str.upper() == str(ticker).upper()) & (self.df['Status'] == 'Open')
        open_trades = self.df[mask].copy()
        if open_trades.empty:
            print(f"❌ No open positions found for {ticker}")
            return False

        remaining = float(sell_quantity)
        for idx, row in open_trades.iterrows():
            if remaining <= 0:
                break
            avail = float(row['Remaining Qty'])
            qty = min(remaining, avail)

            # Update that row
            self.df.at[idx, 'Quantity (Sell)'] = float(self.df.at[idx, 'Quantity (Sell)']) + qty
            self.df.at[idx, 'Sell Price'] = float(sell_price)
            self.df.at[idx, 'Sell Date'] = pd.to_datetime(sell_date)
            self.df.at[idx, 'Sell Value'] = float(self.df.at[idx, 'Quantity (Sell)']) * float(sell_price)
            self.df.at[idx, 'Remaining Qty'] = max(0.0, avail - qty)
            if self.df.at[idx, 'Remaining Qty'] == 0.0:
                self.df.at[idx, 'Status'] = 'Closed'
            remaining -= qty

        self.calculate_mtf_metrics()
        print(f"✅ Sold {sell_quantity} of {ticker} @ ₹{sell_price}")
        return True

    # ---- recompute all metrics (true per-day interest) ----
    def calculate_mtf_metrics(self):
        if self.df.empty:
            return
        today = datetime.now().date()

        for idx, row in self.df.iterrows():
            if pd.isna(row.get('Trade Date')):
                continue

            trade_date = row['Trade Date'].date()
            if row.get('Status') == 'Closed' and not pd.isna(row.get('Sell Date')):
                end_date = row['Sell Date'].date()
            else:
                end_date = today

            days = max(0, (end_date - trade_date).days)
            self.df.at[idx, 'MTF Days Financed'] = int(days)

            # Ensure splits exist
            buy_value = float(row['Buy Value'])
            ucp = float(row.get('User Contribution %', self.default_user_contribution_pct))
            paid = float(row.get('Paid by You', buy_value * (ucp/100.0)))
            financed = float(row.get('MTF Financed Principal', buy_value - paid))
            self.df.at[idx, 'Paid by You'] = paid
            self.df.at[idx, 'MTF Financed Principal'] = financed

            # Day rate (% of principal per day)
            day_rate_pct = float(row.get('MTF Day Rate (%)', 0.04))
            ipd = financed * (day_rate_pct/100.0)           # Interest per day
            self.df.at[idx, 'Interest Per Day'] = float(ipd)
            self.df.at[idx, 'MTF Interest'] = float(ipd * days)

            # P/L
            if row.get('Status') == 'Closed':
                realized = float(self.df.at[idx, 'Sell Value']) - float(self.df.at[idx, 'Quantity (Sell)']) * float(row['Buy Price'])
                self.df.at[idx, 'Realized P/L'] = realized
                self.df.at[idx, 'Unrealized P/L'] = 0.0
            else:
                if row['Remaining Qty'] > 0 and row['Current Price (LTP)'] > 0:
                    unreal = (float(row['Current Price (LTP)']) - float(row['Buy Price'])) * float(row['Remaining Qty'])
                else:
                    unreal = 0.0
                self.df.at[idx, 'Unrealized P/L'] = float(unreal)
                self.df.at[idx, 'Realized P/L'] = 0.0

            total_pl = float(self.df.at[idx, 'Realized P/L']) + float(self.df.at[idx, 'Unrealized P/L'])
            net_pl = total_pl - float(self.df.at[idx, 'MTF Interest'])
            self.df.at[idx, 'Net P/L'] = float(net_pl)
            self.df.at[idx, 'ROI %'] = float((net_pl / buy_value) * 100.0) if buy_value > 0 else 0.0

    # ---- price update ----
    def update_current_prices(self, price_dict):
        for tkr, price in price_dict.items():
            mask = (self.df['Ticker'].str.upper() == str(tkr).upper()) & (self.df['Status'] == 'Open')
            self.df.loc[mask, 'Current Price (LTP)'] = float(price)
        self.calculate_mtf_metrics()
        print("✅ Current prices updated.")

    # ---- portfolio summary ----
    def get_portfolio_summary(self):
        if self.df.empty:
            return {
                'total_trades': 0, 'open_positions': 0, 'closed_positions': 0,
                'total_invested': 0.0, 'total_current_value': 0.0,
                'total_realized_pl': 0.0, 'total_unrealized_pl': 0.0,
                'total_mtf_interest': 0.0, 'total_net_pl': 0.0, 'overall_roi': 0.0
            }

        summary = {
            'total_trades': len(self.df),
            'open_positions': int((self.df['Status'] == 'Open').sum()),
            'closed_positions': int((self.df['Status'] == 'Closed').sum()),
            'total_invested': float(self.df['Buy Value'].sum()),
            'total_current_value': 0.0,
            'total_realized_pl': float(self.df['Realized P/L'].sum()),
            'total_unrealized_pl': float(self.df['Unrealized P/L'].sum()),
            'total_mtf_interest': float(self.df['MTF Interest'].sum()),
            'total_net_pl': float(self.df['Net P/L'].sum()),
            'overall_roi': 0.0
        }

        # current value = open (qty*ltp) + closed (sell value)
        open_pos = self.df[self.df['Status'] == 'Open']
        closed_pos = self.df[self.df['Status'] == 'Closed']
        cv_open = (open_pos['Remaining Qty'] * open_pos['Current Price (LTP)']).sum()
        cv_closed = closed_pos['Sell Value'].sum()
        summary['total_current_value'] = float(cv_open + cv_closed)

        if summary['total_invested'] > 0:
            summary['overall_roi'] = float((summary['total_net_pl'] / summary['total_invested']) * 100.0)
        return summary

    # ---- Groww-style header + table ----
    def groww_style_summary(self):
        """
        Returns:
          header: dict(total_buy_value, paid_by_you, financed, open_interest_per_day)
          table:  DataFrame [Company, Qty, Paid by you, Funded, Interest, Interest/day]
        """
        if self.df.empty:
            return (
                {"total_buy_value":0.0, "paid_by_you":0.0, "financed":0.0, "open_interest_per_day":0.0},
                pd.DataFrame(columns=["Company","Qty","Paid by you","Funded","Interest","Interest/day"])
            )

        df = self.df.copy()
        total_buy = float(df['Buy Value'].sum())
        paid = float(df['Paid by You'].sum())
        financed = float(df['MTF Financed Principal'].sum())
        open_ipd = float(df.loc[df['Status']=='Open', 'Interest Per Day'].sum())

        ag = df.groupby('Ticker').agg({
            'Quantity (Buy)': 'sum',
            'Paid by You': 'sum',
            'MTF Financed Principal': 'sum',
            'MTF Interest': 'sum',
            'Interest Per Day': 'sum'
        }).reset_index().rename(columns={
            'Ticker':'Company',
            'Quantity (Buy)':'Qty',
            'Paid by You':'Paid by you',
            'MTF Financed Principal':'Funded',
            'MTF Interest':'Interest',
            'Interest Per Day':'Interest/day'
        })[["Company","Qty","Paid by you","Funded","Interest","Interest/day"]].sort_values("Interest/day", ascending=False)

        header = {
            "total_buy_value": total_buy,
            "paid_by_you": paid,
            "financed": financed,
            "open_interest_per_day": open_ipd
        }
        return header, ag

# Create a global instance you will reuse
mtf = MTFCalculator(default_user_contribution_pct=40.0)

print("✅ MTFCalculator is ready.")


✅ MTFCalculator is ready.


In [3]:
# STEP 3: Gradio UI (Groww-style summary + full controls)

!pip -q install gradio
import gradio as gr
import pandas as pd
from datetime import datetime
import uuid, os, matplotlib.pyplot as plt

# ---------- helpers ----------
def money(x):
    try: return f"₹{float(x):,.2f}"
    except: return "₹0.00"

def portfolio_summary_as_markdown():
    if mtf.df.empty:
        return "❌ No trades found. Upload CSV or add trades first."
    s = mtf.get_portfolio_summary()
    return "\n".join([
        "## 📊 Portfolio Overview",
        f"- **Total Trades:** {s['total_trades']}",
        f"- **Open Positions:** {s['open_positions']}",
        f"- **Closed Positions:** {s['closed_positions']}",
        f"- **Total Invested:** {money(s['total_invested'])}",
        f"- **Current Portfolio Value:** {money(s['total_current_value'])}",
        "",
        "## 💰 Profit & Loss",
        f"- **Realized P/L:** {money(s['total_realized_pl'])}",
        f"- **Unrealized P/L:** {money(s['total_unrealized_pl'])}",
        f"- **MTF Interest Paid:** {money(s['total_mtf_interest'])}",
        f"- **Net P/L:** {money(s['total_net_pl'])}",
        f"- **Overall ROI:** {s['overall_roi']:.2f}%",
    ])

def open_positions_df():
    if mtf.df.empty: return pd.DataFrame()
    cols = ['Ticker','Quantity (Buy)','Buy Price','Current Price (LTP)',
            'Paid by You','MTF Financed Principal','Interest Per Day',
            'Unrealized P/L','MTF Days Financed','MTF Interest','Net P/L','ROI %','Status']
    return mtf.df.loc[mtf.df['Status']=='Open', cols].reset_index(drop=True)

def closed_positions_df():
    if mtf.df.empty: return pd.DataFrame()
    cols = ['Ticker','Quantity (Sell)','Buy Price','Sell Price','Sell Date',
            'Paid by You','MTF Financed Principal','MTF Interest',
            'Realized P/L','Net P/L','ROI %','Status']
    return mtf.df.loc[mtf.df['Status']=='Closed', cols].reset_index(drop=True)

def full_ledger_df():
    return mtf.df.copy() if not mtf.df.empty else pd.DataFrame()

# ---------- UI actions ----------
def ui_load_csv(file):
    if file is None:
        return "⚠️ Please upload a CSV.", pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), ""
    try:
        ok = mtf.load_data_from_csv(file.name)
        msg = f"✅ Data loaded successfully! {len(mtf.df)} trades found." if ok else "❌ Failed to load CSV."
        return msg, open_positions_df(), closed_positions_df(), full_ledger_df(), portfolio_summary_as_markdown()
    except Exception as e:
        return f"❌ Error reading CSV: {e}", pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), ""

def ui_add_trade(date_str, ticker, qty, buy_price, current_price, mtf_day_rate, user_contribution_pct):
    if not ticker or qty is None or buy_price is None:
        return "⚠️ Fill Ticker, Quantity, Buy Price.", open_positions_df(), full_ledger_df(), portfolio_summary_as_markdown()
    d = date_str if date_str else datetime.now().strftime('%Y-%m-%d')
    mtf.add_new_trade(d, ticker, int(qty), float(buy_price),
                      None if current_price in ("", None) else float(current_price),
                      float(mtf_day_rate),
                      None if user_contribution_pct in ("", None) else float(user_contribution_pct))
    return f"✅ Added {ticker.upper()} x {qty} @ {money(buy_price)}", open_positions_df(), full_ledger_df(), portfolio_summary_as_markdown()

def ui_sell_trade(ticker, sell_qty, sell_price, sell_date):
    if not ticker or sell_qty is None or sell_price is None:
        return "⚠️ Fill Ticker, Quantity, and Sell Price.", open_positions_df(), closed_positions_df(), portfolio_summary_as_markdown()
    mtf.sell_trade(ticker, int(sell_qty), float(sell_price), sell_date if sell_date else None)
    return f"✅ Sold {sell_qty} of {ticker.upper()} @ {money(sell_price)}", open_positions_df(), closed_positions_df(), portfolio_summary_as_markdown()

def ui_update_prices(text):
    """
    Enter: BLACKBUCK:640, HBLENGINE:805, AGARWALEYE:455
    """
    if not text:
        return "⚠️ Enter prices as TICKER:PRICE pairs.", open_positions_df(), portfolio_summary_as_markdown()
    updates = {}
    for pair in text.split(","):
        if ":" in pair:
            k, v = pair.strip().split(":")
            updates[k.strip().upper()] = float(v.strip())
    if not updates:
        return "⚠️ No valid pairs parsed.", open_positions_df(), portfolio_summary_as_markdown()
    mtf.update_current_prices(updates)
    return "✅ Prices updated.", open_positions_df(), portfolio_summary_as_markdown()

def ui_make_charts():
    if mtf.df.empty:
        return "⚠️ No data to visualize.", None
    plt.close('all')
    mtf.create_visualizations() if hasattr(mtf, "create_visualizations") else None
    fig = plt.gcf()
    if not fig:
        return "⚠️ Could not capture figure.", None
    out = f"/content/mtf_charts_{uuid.uuid4().hex[:8]}.png"
    fig.savefig(out, bbox_inches='tight', dpi=140)
    plt.close(fig)
    return "✅ Charts created.", out

def ui_export_csv():
    path = "/content/mtf_portfolio_latest.csv"
    mtf.df.to_csv(path, index=False)
    return path

def ui_groww_summary():
    header, table = mtf.groww_style_summary()
    if table.empty and header["total_buy_value"] == 0:
        return "❌ No data. Upload CSV or add trades.", pd.DataFrame()

    paid_pct = (header["paid_by_you"]/header["total_buy_value"]*100) if header["total_buy_value"] else 0.0
    funded_pct = 100 - paid_pct

    md = f"""
### MTF summary
_All data is computed as of previous day_

**Total buy value:** {money(header['total_buy_value'])}

- **Paid by you:** {money(header['paid_by_you'])}  ({paid_pct:.2f}%)
- **Broker funded:** {money(header['financed'])}  ({funded_pct:.2f}%)

**Open positions interest (per day):** −{money(header['open_interest_per_day'])}
"""
    return md, table

# ---------- UI ----------
with gr.Blocks(title="MTF Trading Calculator") as demo:
    gr.Markdown("# 📈 MTF Trading Calculator")
    gr.Markdown("Upload your CSV or add trades. Includes a **Groww-style MTF summary**.")

    with gr.Tabs():
        # Groww-style Summary
        with gr.Tab("🟡 MTF Summary"):
            sum_btn = gr.Button("Refresh Summary")
            sum_md = gr.Markdown()
            sum_tbl = gr.Dataframe(label="Company-wise", interactive=False)
            sum_btn.click(ui_groww_summary, outputs=[sum_md, sum_tbl])

        # Load CSV
        with gr.Tab("📥 Load CSV"):
            csv_in = gr.File(label="Upload MTF CSV")
            load_btn = gr.Button("Load")
            load_msg = gr.Markdown()
            open_grid = gr.Dataframe(label="Open Positions", interactive=False)
            closed_grid = gr.Dataframe(label="Closed Positions", interactive=False)
            ledger_grid = gr.Dataframe(label="Full Ledger", interactive=False)
            dash_md = gr.Markdown()

            load_btn.click(ui_load_csv, inputs=[csv_in],
                           outputs=[load_msg, open_grid, closed_grid, ledger_grid, dash_md])

        # Add Trade
        with gr.Tab("➕ Add Trade"):
            with gr.Row():
                date_in = gr.Textbox(label="Trade Date (YYYY-MM-DD)", placeholder="Leave blank for today")
                ticker_in = gr.Textbox(label="Ticker", placeholder="e.g., HBLENGINE")
                qty_in = gr.Number(label="Quantity (Buy)", precision=0)
                buy_in = gr.Number(label="Buy Price")
            with gr.Row():
                ltp_in = gr.Textbox(label="Current Price (optional)", placeholder="e.g., 805")
                rate_in = gr.Number(label="MTF Day Rate (%)", value=0.04)
                ucp_in = gr.Number(label="User Contribution % (your margin)", value=40.0)
            add_btn = gr.Button("Add Trade")
            add_msg = gr.Markdown()
            add_open = gr.Dataframe(label="Open Positions (after)", interactive=False)
            add_ledger = gr.Dataframe(label="Full Ledger (after)", interactive=False)
            add_dash = gr.Markdown()

            add_btn.click(ui_add_trade,
                          inputs=[date_in, ticker_in, qty_in, buy_in, ltp_in, rate_in, ucp_in],
                          outputs=[add_msg, add_open, add_ledger, add_dash])

        # Sell
        with gr.Tab("💰 Sell"):
            with gr.Row():
                sell_ticker = gr.Textbox(label="Ticker")
                sell_qty = gr.Number(label="Quantity to Sell", precision=0)
                sell_price = gr.Number(label="Sell Price")
                sell_date = gr.Textbox(label="Sell Date (YYYY-MM-DD, optional)")
            sell_btn = gr.Button("Sell Now")
            sell_msg = gr.Markdown()
            sell_open = gr.Dataframe(label="Open Positions (after)", interactive=False)
            sell_closed = gr.Dataframe(label="Closed Positions (after)", interactive=False)
            sell_dash = gr.Markdown()

            sell_btn.click(ui_sell_trade,
                           inputs=[sell_ticker, sell_qty, sell_price, sell_date],
                           outputs=[sell_msg, sell_open, sell_closed, sell_dash])

        # Update Prices
        with gr.Tab("🔄 Update Prices"):
            price_text = gr.Textbox(label="Ticker:Price pairs",
                                    placeholder="BLACKBUCK:640, HBLENGINE:805, AGARWALEYE:455")
            upd_btn = gr.Button("Update")
            upd_msg = gr.Markdown()
            upd_open = gr.Dataframe(label="Open Positions (after)", interactive=False)
            upd_dash = gr.Markdown()
            upd_btn.click(ui_update_prices, inputs=[price_text], outputs=[upd_msg, upd_open, upd_dash])

        # Dashboard & Charts
        with gr.Tab("📊 Dashboard"):
            dash_btn = gr.Button("Refresh Summary")
            dash_md2 = gr.Markdown()
            chart_btn = gr.Button("Create Charts")
            chart_msg = gr.Markdown()
            chart_img = gr.Image(type="filepath")
            dash_btn.click(lambda: portfolio_summary_as_markdown(), outputs=[dash_md2])
            chart_btn.click(ui_make_charts, outputs=[chart_msg, chart_img])

        # Export
        with gr.Tab("⬇️ Export"):
            exp_btn = gr.Button("Export Current Ledger to CSV")
            exp_file = gr.File(label="Download CSV")
            exp_btn.click(ui_export_csv, outputs=[exp_file])

gr.Markdown("Tip: Start with **📥 Load CSV** or **➕ Add Trade**, then check **🟡 MTF Summary**.")
demo.launch(share=True)  # set to False if you don't want a public link


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://29975dadf5718e7f1a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
